In [2]:
# @title load library dan install dependency
import pandas as pd
import tensorflow as tf
import numpy as np



In [6]:
url_train = "https://raw.githubusercontent.com/apri-oss/paraphrase_generator/main/dataset/data_train.csv"
url_val = "https://raw.githubusercontent.com/apri-oss/paraphrase_generator/main/dataset/data_validation.csv"
url_test = "https://raw.githubusercontent.com/apri-oss/paraphrase_generator/main/dataset/data_test.csv"

data_train = pd.read_csv(url_train)
data_validation = pd.read_csv(url_val)
data_test = pd.read_csv(url_test)

# Tampilkan 5 baris pertama
print(f"Jumlah data Training = {len(data_train)}")
print(f"Jumlah data Validation = {len(data_validation)}")
print(f"Jumlah data Testing = {len(data_test)}")

Jumlah data Training = 2753
Jumlah data Validation = 573
Jumlah data Testing = 574
